In [ ]:
import random, copy
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import re
import time
import json

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class DAGGER_Agent:
    """The DQN agent that interacts with the user."""

    def __init__(self):
        """
        The constructor of DQNAgent.

        The constructor of DQNAgent which saves constants, sets up neural network graphs, etc.

        """
        self.memory = []
        self.num_samples_epoch = 0
        self.memory_index = 0
        self.max_memory_size = 10000 #200000, 125000, 10000,5000? #last stable version 15000
        self.lr = 1e-4 #1e-8, 1e-6, 1e-3
        self.batch_size = 32 #16, 32, 8
        self.hidden_size_1 = 32 #32
        self.hidden_size_2 = 8   #8
        self.model_num = 0
        
        with open('../rule_data.json', 'r') as f:
              self.rule_data = json.load(f)
        #self.len_rule_datapoint = 6

        self.load_weights_file_path = ""
        self.save_weights_file_path = "../dagger_models"

        if self.max_memory_size < self.batch_size:
            raise ValueError('Max memory size must be at least as great as batch size!')

        self.state_size = 26 #the input size to dqn_agent model. 
                             #the size of the state=[actor, hel_action, hel_da, eld_action, eld_da, hel_ot,hel_l,hel_o]
                             #size_state=[1,8,13,6,13,2,2,2] --> 41
                             #the size of the state=[actor, eld_action, eld_da, hel_ot,hel_l,hel_o]
                             #size_state=[1,6,13,2,2,2] --> 26
                        
        self.num_action = 9 #the number of helper's actions 0-8
        self.num_da = 14 #the number of helper's das 0-13


        self.model = self._build_model()
        self.ce = nn.CrossEntropyLoss()
        self.mse = nn.MSELoss()
        self.huber = nn.HuberLoss(reduction='mean', delta=1.0)
        self.mae = nn.L1Loss()

        #self.beh_optimizer = torch.optim.SGD(self.beh_model.parameters(), lr=self.lr, momentum=0.9)
        #self.tar_optimizer = torch.optim.SGD(self.tar_model.parameters(), lr=self.lr, momentum=0.9)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr)

        self._load_weights()



    def _build_model(self):
        """Builds and returns model/graph of neural network."""

        model = helper_model(self.hidden_size_1,self.hidden_size_2, self.state_size,
                             self.num_action).to(device)
        return model


    def get_action(self, agent_input, elder_output, helper_ot_l_o, warmup):
        """
        Returns the action of the agent given a state.

        """
        if warmup:
            return self._rule_action(elder_output, helper_ot_l_o)
        else:
            return self._dagger_action(agent_input)
        
    def get_expert_action(self, agent_input, elder_output, helper_ot_l_o):
        """
        Returns the action of the agent given a state.

        """
        return self._rule_action(elder_output, helper_ot_l_o)


    def prep_input(self, elder_output, helper_ot_l_o):
        helper_input_ot = torch.Tensor(2*[0]).to(device=device)
        helper_input_l = torch.Tensor(2*[0]).to(device=device)        
        helper_input_o = torch.Tensor(2*[0]).to(device=device)
        
        helper_input_actor = torch.Tensor(1*[0]).to(device=device)
        helper_input_eld_action = torch.Tensor(6*[0]).to(device=device)
        helper_input_eld_da = torch.Tensor(13*[0]).to(device=device)

        if helper_ot_l_o[0][1] == 1:
            helper_input_ot[0] = 1
        elif helper_ot_l_o[0][1] == 2:
            helper_input_ot[1] = 1
            
        if helper_ot_l_o[1][1] == 1:
            helper_input_l[0] = 1
        elif helper_ot_l_o[1][1] == 2:
            helper_input_l[1] = 1

        if helper_ot_l_o[2][1] == 1:
            helper_input_o[0] = 1
        elif helper_ot_l_o[2][1] == 2:
            helper_input_o[1] = 1
                
        if elder_output['action_out']:
            helper_input_eld_action[elder_output['action_out']-1] = 1
        else:
            helper_input_actor = torch.Tensor(1*[1]).to(device=device)
            
        if elder_output['da_out']:
            #elder_output['da_out']=elder_output['da_out'].item()
            #if elder_output['da_out']==6 and elder_output['action_out'] in [1,2,3]:
                #elder_output['da_out']= 1
            helper_input_eld_da[elder_output['da_out']-1] = 1

                

        
        helper_input = torch.cat((helper_input_ot, helper_input_l, helper_input_o,
                                  helper_input_actor, helper_input_eld_action, helper_input_eld_da), 0)
        
        return helper_input

    def _rule_action(self, elder_output, helper_ot_l_o):
        #print(elder_output, helper_ot_l_o)
        hel_action_out = {}
        hel_action_out['action_out'] = torch.Tensor(9*[0]).to(device=device)
        indx__ = [0]
        for datapoint in self.rule_data:
            #print(elder_output['action_out'].item(), datapoint['eld_action'])
            if (elder_output['da_out'].item() == datapoint['eld_da'] and
                elder_output['action_out'].item() == datapoint['eld_action'] and
                helper_ot_l_o[0][1] == datapoint['ot'] and helper_ot_l_o[1][1] == datapoint['l'] 
                and helper_ot_l_o[2][1] == datapoint['o']):
                
                if datapoint['hel_action']==2 and datapoint['hel_action']==1:
                    print(datapoint['ot'],datapoint['l'],datapoint['o'])
                    if datapoint['ot']==1 and datapoint['l']==1 and datapoint['o']==0:
                        print(datapoint['hel_action'])
                
                indx__.append(datapoint['hel_action'])
                
        #print(indx__)
        exp_flag = False
        if indx__ != [0]:
            exp_flag = True
        #indx_ = indx__[random.randint(0,len(indx__)-1)]
        indx_ = max(set(indx__), key = indx__.count)
        if len(indx__)>=2 and indx_==0:
            indx_=indx__[1]
            

        hel_action_out['action_out'][indx_] = 1
        #print(hel_action_out)
        return hel_action_out, exp_flag
                
      

    def _dagger_action(self, state):
        """
        Returns a behavior model output given a state.

        """
        agent_output = self._dagger_predict_one(state)
        return agent_output


    def _dagger_predict_one(self, state):
        agent_output = self._dagger_predict(state)
        return agent_output

    def _dagger_predict(self, states):
        """
        Returns a model prediction given an array of states.

        """
        return self.model(states)

    def add_experience(self, state, action, expert_action, done, success):
        """
        Adds an experience made of the parameters to the memory.

        """

        if len(self.memory) < self.max_memory_size:
            self.memory.append(None)
        self.memory[self.memory_index] = (state, action, expert_action, done, success)
        self.memory_index = (self.memory_index + 1) % self.max_memory_size

        
        
    def empty_memory(self):
        """Empties the memory and resets the memory index."""

        self.memory = []
        self.balanced_memory = []
        self.memory_index = 0

    def is_memory_full(self):
        """Returns true if the memory is full."""

        return len(self.memory) == self.max_memory_size

    def train(self):
        """
        Trains the agent by improving the behavior model given the memory tuples.

        Takes batches of memories from the memory pool and processing them. The processing takes the tuples and stacks
        them in the correct format for the neural network and calculates the Bellman equation for Q-Learning.

        """
        # Calc. num of batches to run
        num_batches = len(self.memory) // self.batch_size
        print("num_batches=", num_batches)
        batchlosses_of_oneepoch = [] # each element is the loss of one batch

        for b in range(num_batches):
            batch = random.sample(self.memory, self.batch_size)
            #print('--'*20)
            #for sample in batch:
            #    print(sample)
            #print(len(batch))
            states = [sample[0] for sample in batch]
            actions = [sample[1] for sample in batch]
            expert_actions = [sample[2] for sample in batch]
            

            inputs = []
            labels = []
            

            for i, (s, a, e_a, d, success) in enumerate(batch):
                t = expert_actions[i]                                 
                labels.append(t)                
                inputs.append(self.prep_input(s[0],s[1][1]))


            #das_t = [targets[j]["da_out"] for j in range(len(targets))]
            actions_t = [labels[j]["action_out"] for j in range(len(labels))]
            #print( len(actions_t), actions_t)
            total_train_loss = 0
            self.model.train()            
            for i in range(len(inputs)):                
                to_input = inputs[i]
                to_input = to_input.to(device=device)
                #da_t = das_t[i]
                action_t = actions_t[i]
                
                self.model.zero_grad()
                self.optimizer.zero_grad()
                

                output = self.model(to_input)
                
                loss = torch.zeros(1).to(device=device)
                #loss += self.mse(output["action_out"], action_t.to(device=device))
                #loss += self.mae(output["da_out"], da_t.to(device=device))
                __pred__ = torch.unsqueeze( output["action_out"], dim=0)
                __target__ = torch.as_tensor( torch.argmax( action_t.to(device=device)), device=device).unsqueeze(0)
                loss += self.ce( __pred__, __target__)
                total_train_loss += loss.item()
                loss.backward()
                self.optimizer.step()

            batchlosses_of_oneepoch.append( total_train_loss)

        print("Training loss:", np.mean(np.asarray(batchlosses_of_oneepoch)))
        return batchlosses_of_oneepoch



    def save_weights(self):
        """Saves the weights of both models in two files."""
        if not self.save_weights_file_path:
            return
        save_file_path = self.save_weights_file_path
        #self.beh_model.eval() #??????????????????????????????????????????
        torch.save(self.model.state_dict(), save_file_path + '/dagger_model_'+ str(self.model_num) +'.pt')
        self.load_weights_file_path = self.save_weights_file_path


    def _load_weights(self):
        """Loads the weights of both models from two h5 files."""

        if not self.load_weights_file_path:
            return
        load_file_path = self.load_weights_file_path
        self.model.load_state_dict(torch.load(load_file_path + '/dagger_model.pt'))
        self.model.to(device=device)
        #self.beh_model.eval()

        

class helper_model(nn.Module):
    def __init__(self, size1, size2, num_in, num_action_out):
        super(helper_model, self).__init__()
        self._size1 = size1
        self._size2 = size2
        self._layer1 = nn.Linear(num_in, size1)
        self._layer2 = nn.Linear(size1, size2)
        self._dropout = nn.Dropout(p=0.2)
        #self._da_out = nn.Linear(size2, num_da_out)
        self._action_out = nn.Linear(size2, num_action_out)

    def forward(self, feature_input):
        layer1_out = self._layer1(feature_input)
        layer2_out = self._layer2(layer1_out)
        hidden_out = self._dropout(layer2_out)
        to_return = {}
        #to_return["da_out"] = F.relu(self._da_out(hidden_out))
        to_return["action_out"] = F.relu(self._action_out(hidden_out))
        return to_return